In [ ]:
from urllib.parse import urlencode
from pyquery import PyQuery as py
import requests
import pandas as pd

In [ ]:
def get_information(since_id = '',uid = 0,):
    #X-Requested-With 用来标识Ajax请求，必须得有
    #Referer 用来指明请求来源  必须有
    #User-Agent  伪装浏览器，必须有
    headers = {'Referer': 'https://m.weibo.cn',
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36',
                'X-Requested-With': 'XMLHttpRequest'}

    params = {
                'uid': uid,
                'luicode': 10000011,
                'lfid': '231093_-_selffollowed',
                'type': int(uid),
                'value': 6620842908,
                'containerid': '107603' + str(uid),
                'since_id': since_id
    }

    # urlencode() 方法将参数转化为URL的GET请求参数
    url = 'https://m.weibo.cn/api/container/getIndex?' + urlencode(params)
    response = requests.get(url,headers = headers)
    #获得返回的 json 内容，做分析
    json_text = response.json()
    #获得 since_id 为增加页面做准备
    since_id = json_text.get('data').get('cardlistInfo').get('since_id')
    return json_text,since_id

In [ ]:
def parse_json(json):
    items = json.get('data').get('cards')


    for item in items:
        item = item.get('mblog')
        weibo = {}
        weibo['发表时间'] = item.get('created_at')
        #weibo['手机类型'] = item.get('source')
        weibo['转发数量'] = item.get('reposts_count')
        #weibo['图片链接'] = item.get('bmiddle_pic')
        weibo['点赞数'] = item.get('attitudes_count')
        weibo['评论数'] = item.get('comments_count')
        weibo['内容'] = py(item.get('text')).text()
        yield weibo

In [ ]:

#uid 你所要爬取的微博的ID，在响应的参数列表中可以得到，图中可以找到
uid = 6620842908
#p 爬取的页数
p = 10
data = []
s = get_information(since_id = 4595747292721435,uid = uid)
print(s[1])
for i in parse_json(s[0]):
    print(i)
    data.append(i)
#解析 JSON
#parse_json(s[0])
#输出解析后的内容
'''
for i in parse_json(s[0]):
    print(i)
'''
#多页爬取

for i in range(p):
    s = get_information(since_id = s[1],uid = uid)
    print(s[1])
    for j in parse_json(s[0]):
        print(j)
        data.append(j)

pd.DataFrame(data).to_csv('tnt20211.csv',encoding="utf_8_sig")